## Data Cleaning - Pre-Generated Data Files  
### EPA - United States Environmental Protection Agency  

**Link:** [EPA Air Data Download Files](https://aqs.epa.gov/aqsweb/airdata/download_files.html)


In [1]:
import pandas as pd

In [24]:
import pandas as pd

def cleanData(year: str):
    # change to respective path
    df = pd.read_csv(f'../data/raw-data/annual_conc_by_monitor/individual-data/annual_conc_by_monitor_{year}.csv', low_memory=False)
    #Ozone Data Cleaning and Processing
    filtered_Ozone_df = df[(df['Parameter Name'] == 'Ozone') & (df['Sample Duration'] == '8-HR RUN AVG BEGIN HOUR')]

    # Calculating the average of 'Arithmetic Mean' grouped by 'State Code'

    ozone_average_by_state = filtered_Ozone_df.groupby('State Name')['Arithmetic Mean'].mean().reset_index(name='Ozone Average')

    filtered_Sulfur_df = df[(df['Parameter Name'] == 'Sulfur dioxide') & (df['Sample Duration'] == '1 HOUR')]

    #Calculating the average of 'Arithmetic Mean' grouped by 'State Code'
    sulfur_average_by_state = filtered_Sulfur_df.groupby('State Name')['Arithmetic Mean'].mean().reset_index(name='Sulfur Average')

    filtered_Carbon_df = df[(df['Parameter Name'] == 'Carbon monoxide') & (df['Sample Duration'] == '8-HR RUN AVG END HOUR')]

    # Calculating the average of 'Arithmetic Mean' grouped by 'State Code'
    carbon_average_by_state = filtered_Carbon_df.groupby('State Name')['Arithmetic Mean'].mean().reset_index(name='Carbon Average')

    filtered_Nitrogen_df = df[(df['Parameter Name'] == 'Nitrogen dioxide (NO2)') & (df['Sample Duration'] == '1 HOUR')]

    # Calculating the average of 'Arithmetic Mean' grouped by 'State Code'
    nitrogen_average_by_state = filtered_Nitrogen_df.groupby('State Name')['Arithmetic Mean'].mean().reset_index(name='Nitrogen Average')

    filtered_Small_df = df[(df['Parameter Name'] == 'PM2.5 - Local Conditions') & (df['Sample Duration'] == '24-HR BLK AVG')]

    # Calculating the average of 'Arithmetic Mean' grouped by 'State Code'
    small_particulate_average_by_state = filtered_Small_df.groupby('State Name')['Arithmetic Mean'].mean().reset_index(name='PM2.5 Average')

    filtered_df = df[(df['Parameter Name'] == 'PM10 Total 0-10um STP') & (df['Sample Duration'] == '24-HR BLK AVG')]

    # Calculating the average of 'Arithmetic Mean' grouped by 'State Code'
    large_particulate_average_by_state = filtered_df.groupby('State Name')['Arithmetic Mean'].mean().reset_index(name='PM10 Average')

    all_cols = df.columns.tolist()
    selected_cols = [   'Year','State Name',
                     "Ozone Average","Carbon Average",	"Nitrogen Average",	
                     "Sulfur Average",	"PM2.5 Average","PM10 Average",]
    
    remove_cols = list(set(all_cols) - set(selected_cols))
    df.drop(remove_cols, axis=1, inplace=True)

    df.drop_duplicates(inplace=True, ignore_index=True)

    df = df.merge(ozone_average_by_state, on='State Name', how='left')
    df = df.merge(carbon_average_by_state, on='State Name', how='left')
    df = df.merge(nitrogen_average_by_state, on='State Name', how='left')
    df = df.merge(sulfur_average_by_state, on='State Name', how='left')
    df = df.merge(small_particulate_average_by_state, on='State Name', how='left')
    df = df.merge(large_particulate_average_by_state, on='State Name', how='left')
    

    return df
    



In [21]:
# Ozone filteration and average calulation by state

df = pd.read_csv(f'../data/raw-data/annual_conc_by_monitor/individual-data/annual_conc_by_monitor_{year}.csv', low_memory=False)

filtered_df = df[(df['Parameter Name'] == 'Ozone') & (df['Sample Duration'] == '8-HR RUN AVG BEGIN HOUR')]

# Calculating the average of 'Arithmetic Mean' grouped by 'State Code'
#ozone_average_by_state = filtered_df.groupby('State Name')['Arithmetic Mean'].mean()
ozone_average_by_state = filtered_df.groupby('State Name')['Arithmetic Mean'].mean().reset_index(name='Ozone Average')

#df = df.merge(ozone_average_by_state, on='State Name', how='left')

#df
ozone_average_by_state
# Display the result
#print(ozone_average_by_state)

,State Name,Ozone Average
0,Alabama,0.043222
1,Alaska,0.028366
2,Arizona,0.049037
3,Arkansas,0.041512
4,California,0.042853
5,Colorado,0.049375
6,Connecticut,0.044706
7,Country Of Mexico,0.042174
8,Delaware,0.044349
9,District Of Columbia,0.039905


In [ ]:
#Sulphur averagecalulcation by state

df = pd.read_csv(f'/Users/rahib/ctp_data/annual_conc_by_monitor_2023.csv', low_memory=False)

filtered_df = df[(df['Parameter Name'] == 'Sulfur dioxide') & (df['Sample Duration'] == '1 HOUR')]

# Calculating the average of 'Arithmetic Mean' grouped by 'State Code'
sulfur_average_by_state = filtered_df.groupby('State Name')['Arithmetic Mean'].mean().reset_index(name='Sulfur Average')


# Display the result
print(sulfur_average_by_state)

In [ ]:
#Carbon Average Calculation by State

df = pd.read_csv(f'/Users/rahib/ctp_data/annual_conc_by_monitor_2023.csv', low_memory=False)

filtered_df = df[(df['Parameter Name'] == 'Carbon monoxide') & (df['Sample Duration'] == '8-HR RUN AVG END HOUR')]

# Calculating the average of 'Arithmetic Mean' grouped by 'State Code'
carbon_average_by_state = filtered_df.groupby('State Name')['Arithmetic Mean'].mean().reset_index(name='Carbon Average')

# Display the result
print(carbon_average_by_state)

In [ ]:
#Nitrogen Average Calculation by State

df = pd.read_csv(f'/Users/rahib/ctp_data/annual_conc_by_monitor_2023.csv', low_memory=False)

filtered_df = df[(df['Parameter Name'] == 'Nitrogen dioxide (NO2)') & (df['Sample Duration'] == '1 HOUR')]

# Calculating the average of 'Arithmetic Mean' grouped by 'State Code'
nitrogen_average_by_state = filtered_df.groupby('State Name')['Arithmetic Mean'].mean().reset_index(name='Nitrogen Average')

# Display the result
print(nitrogen_average_by_state)

In [ ]:
#PM2.5 Average Calculation by State

df = pd.read_csv(f'/Users/rahib/ctp_data/annual_conc_by_monitor_2023.csv', low_memory=False)

filtered_df = df[(df['Parameter Name'] == 'PM2.5 - Local Conditions') & (df['Sample Duration'] == '24-HR BLK AVG')]

# Calculating the average of 'Arithmetic Mean' grouped by 'State Code'
small_particulate_average_by_state = filtered_df.groupby('State Name')['Arithmetic Mean'].mean().reset_index(name='PM2.5 Average')

# Display the result
print(small_particulate_average_by_state)

In [ ]:
#PM10 Average Calculation by State

df = pd.read_csv(f'/Users/rahib/ctp_data/annual_conc_by_monitor_2023.csv', low_memory=False)

filtered_df = df[(df['Parameter Name'] == 'PM10 Total 0-10um STP') & (df['Sample Duration'] == '24-HR BLK AVG')]


# Calculating the average of 'Arithmetic Mean' grouped by 'State Code'
large_particulate_average_by_state = filtered_df.groupby('State Name')['Arithmetic Mean'].mean().reset_index(name='PM10 Average')

# Display the result
print(large_particulate_average_by_state)

In [ ]:
df = df.merge(ozone_average_by_state, on='State Name', how='left')
df = df.merge(carbon_average_by_state, on='State Name', how='left')
df = df.merge(nitrogen_average_by_state, on='State Name', how='left')
df = df.merge(sulfur_average_by_state, on='State Name', how='left')
df = df.merge(small_particulate_average_by_state, on='State Name', how='left')
df = df.merge(large_particulate_average_by_state, on='State Name', how='left')
df

In [63]:
all_cols = df.columns.tolist()
selected_cols = [   'State Name',
                     "Ozone Average","Carbon Average",	"Nitrogen Average",	
                     "Sulfur Average",	"PM2.5 Average","PM10 Average",]
    
remove_cols = list(set(all_cols) - set(selected_cols))
df.drop(remove_cols, axis=1, inplace=True)

df.drop_duplicates(inplace=True, ignore_index=True)


In [ ]:
df.drop_duplicates(inplace=True, ignore_index=True)
df

In [ ]:
#Breakpoints

pm25_breakpoints = {
    (0.0, 12.0): (0, 50),
    (12.1, 35.4): (51, 100),
    (35.5, 55.4): (101, 150),
    (55.5, 150.4): (151, 200),
    (150.5, 250.4): (201, 300),
    (250.5, 350.4): (301, 400),
    (350.5, 500.4): (401, 500)
}
pm10_breakpoints = {
    (0.0, 54.9): (0, 50),
    (55, 154.9): (51, 100),
    (155, 254.9): (101, 150),
    (255, 354.9): (151, 200),
    (355, 424.9): (201, 300),
    (425, 504.9): (301, 400),
    (505, 609): (401, 500)
}

no2_breakpoints = {
    (0, 53): (0, 50),
    (54, 100): (51, 100),
    (101, 360): (101, 150),
    (361, 649): (151, 200),
    (650, 1249): (201, 300),
    (1250, 1649): (301, 400),
    (1650, 2049): (401, 500)
}
ozone_breakpoints = {
    (0, 54): (0, 50),
    (55, 70): (51, 100),
    (71, 85): (101, 150),
    (86, 105): (151, 200),
    (106, 200): (201, 300)
}
carbon_breakpoints = {
    (0, 4.4): (0, 50),
    (4.5, 9.4): (51, 100),
    (9.5, 12.4): (101, 150),
    (12.5, 15.4): (151, 200),
    (15.5, 30.4): (201, 300)
}

sulphur_breakpoints = {
    (0, 35.9): (0, 50),
    (36, 76.9): (51, 100),
    (77, 185.9): (101, 150),
    (186, 304.9): (151, 200),
    (305, 604.9): (201, 300)
}

def calculate_aqi_from_concentration(concentration, breakpoints):
    for (bp_lo, bp_hi), (aqi_lo, aqi_hi) in breakpoints.items():
        if bp_lo <= concentration <= bp_hi:
            return ((aqi_hi - aqi_lo) / (bp_hi - bp_lo)) * (concentration - bp_lo) + aqi_lo
    # Return maximum AQI value otherwise
    return 500

# Truncate the concentration values according to the rules
df['Fine particles (PM 2.5)'] = df['Fine particles (PM 2.5)'].apply(lambda x: round(x, 1))
df['Nitrogen dioxide (NO2)'] = df['Nitrogen dioxide (NO2)'].apply(lambda x: round(x))
df['Ozone (O3)'] = df['Ozone (O3)'].apply(lambda x: round(x))

# Calculate the AQI for each pollutant
df['PM2.5 AQI'] = df['Fine particles (PM 2.5)'].apply(lambda x: calculate_aqi_from_concentration(x, pm25_breakpoints))
df['NO2 AQI'] = df['Nitrogen dioxide (NO2)'].apply(lambda x: calculate_aqi_from_concentration(x, no2_breakpoints))
df['Ozone AQI'] = df['Ozone (O3)'].apply(lambda x: calculate_aqi_from_concentration(x, ozone_breakpoints))

# The overall AQI for the date would be the maximum AQI value across all pollutants
AQdf['AQI'] = df[['PM2.5 AQI', 'NO2 AQI', 'Ozone AQI']].mean(axis=1).round().astype(int)


In [22]:
#Define the years as strings
years = ["2000","2001","2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021","2022"]

In [27]:
#Global DF
df = pd.DataFrame()

for year in years:
    partial_df = cleanData(year)

    df = pd.concat([df, partial_df])

    
# change to respective path
df.to_csv(f'../Processed_Data/AQI_Ratings.csv', index=False)